In [1]:
n_exp = 5

In [2]:
# Experimento 2 
#EN_EMBEDDING_DIM = 128
#ZH_EMBEDDING_DIM = 128
#HIDDEN_DIM = 1024
#CUDA_DEVICE = 0

# Experimento 3 
#EN_EMBEDDING_DIM = 128
#ZH_EMBEDDING_DIM = 128
#HIDDEN_DIM = 256
#CUDA_DEVICE = 0

# Educativo

In [3]:
#Shit
!python training.py --name_file=Educativo/educativo  --dir_files=ParallelCorpora --dir_results=results_educativo_{n_exp}/bpe_5000 --evaluate=False --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
4768it [00:03, 1299.29it/s]
607it [00:00, 1387.25it/s]
607it [00:00, 1353.52it/s]
100%|████████████████████████████████████| 5375/5375 [00:00<00:00, 83500.81it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 5.3632 ||: 100%|██████████████████████████| 48/48 [00:08<00:00,  6.12it/s]
BLEU: 0.0000, loss: 5.0829 ||: 100%|██████████████| 7/7 [00:01<00:00,  6.36it/s]
loss: 4.9151 ||: 100%|██████████████████████████| 48/48 [00:08<00:00,  6.08it/s]
BLEU: 0.0000, loss: 4.9486 ||: 100%|██████████████| 7/7 [00:01<00:00,  5.97it/s]
loss: 4.8282 ||: 100%|██████████████████████████| 48/48 [00:08<00:00,  5.19it/s]
BLEU: 0.0010, loss: 4.8587 ||: 100%|██████████████| 7/7 [00:00<00:00,  7.11it/s]
loss: 4.6473 ||: 100%|██████████████████████████| 48/48 [00:08<00:00,  5.47it/s]
BLEU: 0.0000, loss: 4.6666 ||: 100%|██████████████| 7/7 [00:01<00:00,  5.66i

In [8]:

import csv
from typing import Dict
import logging

from overrides import overrides

from allennlp.common.checks import ConfigurationError
from allennlp.common.file_utils import cached_path
from allennlp.common.util import START_SYMBOL, END_SYMBOL
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import TextField
from allennlp.data.instance import Instance
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer

logger = logging.getLogger(__name__)  # pylint: disable=invalid-name


class CustomSeq2SeqDatasetReader(DatasetReader):
    """
    Read a tsv file containing paired sequences, and create a dataset suitable for a
    ``SimpleSeq2Seq`` model, or any model with a matching API.
    Expected format for each input line: <source_sequence_string>\t<target_sequence_string>
    The output of ``read`` is a list of ``Instance`` s with the fields:
        source_tokens: ``TextField`` and
        target_tokens: ``TextField``
    `START_SYMBOL` and `END_SYMBOL` tokens are added to the source and target sequences.
    Parameters
    ----------
    source_tokenizer : ``Tokenizer``, optional
        Tokenizer to use to split the input sequences into words or other kinds of tokens. Defaults
        to ``WordTokenizer()``.
    target_tokenizer : ``Tokenizer``, optional
        Tokenizer to use to split the output sequences (during training) into words or other kinds
        of tokens. Defaults to ``source_tokenizer``.
    source_token_indexers : ``Dict[str, TokenIndexer]``, optional
        Indexers used to define input (source side) token representations. Defaults to
        ``{"tokens": SingleIdTokenIndexer()}``.
    target_token_indexers : ``Dict[str, TokenIndexer]``, optional
        Indexers used to define output (target side) token representations. Defaults to
        ``source_token_indexers``.
    source_add_start_token : bool, (optional, default=True)
        Whether or not to add `START_SYMBOL` to the beginning of the source sequence.
    delimiter : str, (optional, default="\t")
        Set delimiter for tsv/csv file.
    """
    def __init__(self,
                 vocab,
                 source_tokenizer: Tokenizer = None,
                 target_tokenizer: Tokenizer = None,
                 source_token_indexers: Dict[str, TokenIndexer] = None,
                 target_token_indexers: Dict[str, TokenIndexer] = None,
                 source_add_start_token: bool = True,
                 delimiter: str = "\t",
                 lazy: bool = False) -> None:
        super().__init__(lazy)
        self._vocab_in = vocab.get_token_to_index_vocabulary('tokens')
        self._vocab_out = vocab.get_token_to_index_vocabulary('target_tokens')
        self._source_tokenizer = source_tokenizer or WordTokenizer()
        self._target_tokenizer = target_tokenizer or self._source_tokenizer
        self._source_token_indexers = source_token_indexers or {"tokens": SingleIdTokenIndexer()}
        self._target_token_indexers = target_token_indexers or self._source_token_indexers
        self._source_add_start_token = source_add_start_token
        self._delimiter = delimiter

    @overrides
    def _read(self, file_path):
        with open(cached_path(file_path), "r") as data_file:
            logger.info("Reading instances from lines in file at: %s", file_path)
            for line_num, row in enumerate(csv.reader(data_file, delimiter=self._delimiter)):
                if len(row) != 2:
                    raise ConfigurationError("Invalid line format: %s (line number %d)" % (row, line_num + 1))
                source_sequence, target_sequence = row
                source_sequence = ' '.join([word for word in source_sequence.split() if word in self._vocab_in])
                target_sequence = ' '.join([word for word in target_sequence.split() if word in self._vocab_out])
                
                yield self.text_to_instance(source_sequence, target_sequence)

    @overrides
    def text_to_instance(self, source_string: str, target_string: str = None) -> Instance:  # type: ignore
        # pylint: disable=arguments-differ
        tokenized_source = self._source_tokenizer.tokenize(source_string)
        if self._source_add_start_token:
            tokenized_source.insert(0, Token(START_SYMBOL))
        tokenized_source.append(Token(END_SYMBOL))
        source_field = TextField(tokenized_source, self._source_token_indexers)
        if target_string is not None:
            tokenized_target = self._target_tokenizer.tokenize(target_string)
            tokenized_target.insert(0, Token(START_SYMBOL))
            tokenized_target.append(Token(END_SYMBOL))
            target_field = TextField(tokenized_target, self._target_token_indexers)
            return Instance({"source_tokens": source_field, "target_tokens": target_field})
        else:
            return Instance({'source_tokens': source_field})

In [15]:
# coding: utf-8

import itertools
from typing import Iterator, List, Dict
import fire
from allennlp.common.util import prepare_environment
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.fields import TextField, IndexField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data import Instance

from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.models import DecomposableAttention
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer
from allennlp.data.dataset_readers import DatasetReader
from allennlp.training.metrics.bleu import BLEU
from allennlp.common.params import Params

prepare_environment(Params({}))

import random
import torch
import torch.optim as optim
import nltk
import pandas as pd
import numpy as np
import os 
from pathlib import Path
import warnings

EN_EMBEDDING_DIM = 128
ZH_EMBEDDING_DIM = 128
CUDA_DEVICE = 0

def main(name_file='Educativo/educativo', dir_files='ParallelCorpora', dir_results='results_educativo_4/bpe_5000', cuda_id=0, hidden_dim=1024, evaluate=True, bpe=False):
    
    dir_train = os.path.join(dir_files, name_file)
    dir_test = os.path.join(dir_files, name_file)
    os.makedirs(dir_results, exist_ok=True)
    
    reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=WordTokenizer(),
        delimiter='\t',
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
    
    if bpe:
        train_dataset = reader.read(os.path.join(dir_files, name_file + '_train.bpe.tsv'))
        validation_dataset = reader.read(os.path.join(dir_files, name_file + '_val.bpe.tsv'))
        test_dataset = reader.read(os.path.join(dir_files, name_file + '_test.bpe.tsv'))
    else:
        train_dataset = reader.read(os.path.join(dir_files, name_file + '_train.tsv'))
        validation_dataset = reader.read(os.path.join(dir_files, name_file + '_val.tsv'))
        test_dataset = reader.read(os.path.join(dir_files, name_file + '_test.tsv'))
    
    vocab = Vocabulary.from_instances(train_dataset + validation_dataset,
                                      min_count={'tokens': 8, 'target_tokens': 8})

    en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                             embedding_dim=EN_EMBEDDING_DIM)
    print('size of vocab:', vocab)

    encoder = PytorchSeq2SeqWrapper(torch.nn.GRU(EN_EMBEDDING_DIM, hidden_dim, dropout=0.25, num_layers=2, bidirectional=True, batch_first=True))

    source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})
    # attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
    # attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
    attention = DotProductAttention()

    max_decoding_steps = 100   # TODO: make this variable
    model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                          target_embedding_dim=ZH_EMBEDDING_DIM,
                          target_namespace='target_tokens',
                          attention=attention,
                          beam_size=1,
                          use_bleu=True).cuda()
    
    optimizer = optim.Adam(model.parameters())
    iterator = BucketIterator(batch_size=100, sorting_keys=[("source_tokens", "num_tokens")])

    iterator.index_with(vocab)

    if not evaluate:
        trainer = Trainer(model=model,
                          optimizer=optimizer,
                          iterator=iterator,
                          train_dataset=train_dataset,
                          #validation_metric='+BLEU',
                          validation_dataset=validation_dataset,
                          num_epochs=40,
                          serialization_dir=dir_results,
                          cuda_device=cuda_id)

        trainer.train()
        
        
        
    reader = CustomSeq2SeqDatasetReader(
        vocab,
        source_tokenizer=WordTokenizer(),
        target_tokenizer=WordTokenizer(),
        delimiter='\t',
        source_token_indexers={'tokens': SingleIdTokenIndexer('source_tokens')},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
    
    if bpe:
        test_dataset = reader.read(os.path.join(dir_files, name_file + '_test.bpe.tsv'))
    else:
        test_dataset = reader.read(os.path.join(dir_files, name_file + '_test.tsv'))

    with open(os.path.join(dir_results, "best.th"), 'rb') as f:
        model.load_state_dict(torch.load(f))
    model.eval()
    predictor = SimpleSeq2SeqPredictor(model, reader)
    total_bleu = 0
    len_test = 0
    for instance in test_dataset:

        hypothesis = (' '.join(predictor.predict_instance(instance)['predicted_tokens'])).replace('@@UNKNOWN@@', '').replace('@@ ', '').split()
        reference = (' '.join([token.text for token in instance['target_tokens'].tokens[1:-1]]).replace('@@ ', '')).split()
        print(instance['source_tokens'].tokens[1:-1])
        print(hypothesis)
        print(reference)
        print()
        
        if len(reference) and len(instance['source_tokens'].tokens[1:-1]):
            len_test += 1

            total_bleu += nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    
    print('=================================')
    print('BLEU test:', total_bleu / len_test)
                                                                            

In [16]:
main(dir_results='results_educativo_4/bpe_5000')

4768it [00:04, 1190.00it/s]
607it [00:00, 1359.24it/s]
100%|██████████| 5375/5375 [00:00<00:00, 80573.21it/s]


size of vocab: Vocabulary with namespaces:
 	Non Padded Namespaces: {'*labels', '*tags'}
 	Namespace: tokens, Size: 797 
 	Namespace: target_tokens, Size: 581 



607it [00:00, 1639.90it/s]


[algunas, letras, una, silaba, y, en, otras, hay, letra]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jatiribi', 'jake', 'westiora', 'itan', 'jake', 'jawen', 'betan', 'wisha']

[recoge, los, saberes, previos, de, los, ninos, y, las, ninas, tarjetas, con, imagenes, de, objetos, segun, el, grosor]
['yoyo', 'ikanai', 'ja', 'non']
['biwe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebaon', 'onan', 'shinanbo', 'tarjeta', 'jawen', 'oinxon']

[reflexiona, con, los, ninos, y, las, ninas, con, ellos, sobre, la, ubicacion, de, los, objetos, del, aula, y, la, representacion, que, hicieron, utilizando, las, nociones, dentro, y, fuera]
['yoyo', 'ikanai', 'westiora']
['jato', 'shinankinwe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo', 'axetiainko', 'jatibi', 'jan', 'axeti', 'jakon', 'akin', 'ixon', 'itanribi', 'ja', 'akanabo', 'itan', 'ixon']

[pregunta, a, los, ninos, y, a, las, ninas]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['yokawe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo']

[los, ninos, y, l

[prepara, tiras, de, papel, con, los, nombres, de, los, grupos, del, cuaderno, de, trabajo, y, el, libro, comunicacion]
['yoyo', 'ikanai', 'ja', 'non']
['kirikabo', 'wishawe', 'ja', 'janeyabo', 'jan', 'teeti', 'kirikainoa', 'itan', 'yoyo', 'ati', 'kirika']

[se, deben, observar, algunos, porque, los, ninos, y, las, ninas, han, a]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jake', 'wetsa', 'keskaakin', 'akana', 'benbo', 'bake', 'itan', 'ainbo', 'bakebaon', 'kopi']

[para, a, los, ninos, y, ninas, con, la, y, la, de, la, escritura]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jaskaaxon', 'benbo', 'bakebo', 'itan', 'bakebo', 'keskaakin', 'itan', 'akin', 'wishakanti']

[que, contar, y, comparar, cantidades, de, hasta, tres, cifras, ,, dibujos, con, representacion, de, los, numeros]
['yoyo', 'ikanai', 'ja', 'non']
['neno', 'riki', 'jaskaaxon', 'topontibo', 'akana', 'itanribi', 'oinkanti', 'jisa', 'kimisha', 'akana', ',', 'ribojobo', 'ja', 'oinxon', 'akanabo', ',']

[para, que, las, partes, d

...

[uno, de, los, retos, que, se, les, presenta, en, esta, unidad, comprende, la, siguiente, pregunta]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jaskatax', 'bakebo', 'jakoni', 'axekanti', 'kopi', 'non', 'onanti', 'riki', 'nato', 'unidain', 'nato', 'jawekibo']

[del, sistema]
['yoyo', 'ikanai', 'ja', 'shinan']
['jaskatax', 'axeti', 'jaska', 'akin', 'ati']

[plantea, la, siguiente, situacion]
['yoyo', 'ikanai', 'ja', 'shinan']
['yoiwe', 'ja', 'joai', 'jaweki']

[¿, como, podemos, escribir, el, acuerdo, para, que, todosas, lo, mejor, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'jawekeska', 'axonmein', 'non', 'ati', 'atipanke', 'shinan', 'akin', 'jaskatax', 'noa', 'jakon', 'kopi', '?']

[cuando, ustedes, al, aula]
['yoyo', 'ikanai', 'ja', 'shinan']
['jawetianki', 'maton', 'axetiainko', ',']

[el, proposito, de, la, sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['jato', 'jawe', 'ikarin', 'nato', 'ixon']

[prepara, tiras, de, papel, o, cartulina, de, colores, para, los, letreros, de, los,

[solo, o, por, medio, del]
['yoyo', 'ikanai', 'ja', 'shinan']
['iamax', 'westiora', 'akina']

[sector, de]
['yoyo', 'ikanai', 'ja', 'shinan']
['jainxon', 'mesko', 'jawekibo', 'non', 'onanti']

[usamos, el, lenguaje, oral, y, escrito, para, compartir, como, fue, nuestro, primer, dia, de, clase, !]
['yoyo', 'ikanai', 'ja', 'non']
['nonra', 'yoyo', 'ixon', 'wishai', 'jatixonbi', 'onankanti', 'kopi', 'jawekeskaki', 'non', 'reken', 'axeti', 'neten', 'noa', 'winota', 'ixon', '!']

[tambien, puedes, dos, hojas, para, lograr, hojas, de, un, tamano, mas, grande]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jaskaribi', 'akin', 'taxnanti', 'atipanke', 'rabe', 'pei', 'kirika', 'jaskaaxon', 'ani', 'akin', 'ati', 'kopi']

[las, cajas]
['yoyo', 'ikanai', 'ja', 'shinan']
['jain', 'jaweki']

[dibujen, tantos, para]
['yoyo', 'ikanai', 'ja', 'shinan']
['icha', 'jaskaaxon']

[expresa, el, criterio, para, ordenar]
['yoyo', 'ikanai', 'ja', 'shinan']
['shinanai', 'jaskaakin', 'jawekibo']

[luego, pide, que, a

[¿, donde, lo, vamos, a, escribir, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'jaweranoki', 'non', 'wishai', 'kai', '?']

[¿, donde, lo, ?]
['yoyo', 'ikanai', 'ja', 'shinan']
['¿', 'jaweranomein', 'non', 'kai', '?']

[si, los, textos, han, escritos, en, nivel, alfabetico]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['oinwe', 'akana', 'ixon', 'ja', 'wishabo']

[capacidad, e, a, trabajar, en, la, sesion]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['jaton', 'atipana', 'kaman', 'akanai', 'onankia', 'ixon', 'non', 'jaskatax', 'nato', 'axetiainkoniax', 'teetai']

[organiza, a, los, estudiantes, en, cuatro, grupos, y, formula, las, siguientes, preguntas]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['bakebo', 'tsinkiwe', 'chosko', 'tsama', 'axon', 'jainxon', 'mesko', 'yokabo', 'jato']

[plantea, las, siguientes]
['yoyo', 'ikanai', 'ja', 'shinan']
['yoiwe', 'ja', 'joai', 'yokabo']

[¿, que, les, a, completar, o, mejorar, lo, que, escrito, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'm

[el, destinatario]
['yoyo', 'ikanai', 'ja', 'shinan']
['jain', 'kai']

[]
['icha', 'jan', 'bakebo']
['wetsa', 'jonibo', 'jaton', ',']

[¿, que, haciendo, con, el, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'jawe', 'mato', '?']

[escribe, carteles, en, el, nivel, alfabetico, para, el, croquis, que, elabora, en, situaciones, comunicativas]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['ani', 'wishabo', 'awe', 'nivel', 'alfabetikoain', 'jaskaaxon', 'croquis', 'atinko']

[que, mientras, a, otras, parejas]
['yoyo', 'ikanai', 'ja', 'shinan']
['jato', 'abakanon', 'shinankin', 'min', 'wetsa', 'bakebo', 'oinai', 'kaman']

[pide, a, cada, grupo, que, las, necesarias, de, acuerdo, con, las, que]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['tsinkibo', 'yokawe', 'jakon', 'akanti', 'min', 'shinan', 'jato']

[¿, como, que, se, vea, tu, salon, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'maton', 'oinkasai', 'jainoax', 'mato', 'axeai', '?']

[asi, como, en, su, comprension, de, mensajes, ora

[dentro, o, fuera, del, aula]
['yoyo', 'ikanai', 'ja', 'shinan']
['axeti', 'chicho', 'iamax', 'jeman']

[sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['tsinkiti']

[sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['tsinkiti']

[sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['tsinkiti']

[sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['tsinkiti']

[sesion]
['yoyo', 'ikanai', 'ja', 'shinan']
['tsinkiti']

[tambien, puedes, usar, una, mas, que, la, anterior]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['min', 'atipanke']

[los, ninos, pueden, que]
['yoyo', 'ikanai', 'ja', 'shinan']
['bakebaonra', 'ati', 'atipanke', 'mia', 'yokakin', 'min']

[puede, ser, el]
['yoyo', 'ikanai', 'ja', 'shinan']
['iti', 'atipanke']

[escribe, textos, diversos, en, nivel, alfabetico]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['wishawe', 'meskokeska', 'wishabo']

[¿, a, quien, han, sus, mensajes, ?]
['yoyo', 'ikanai', 'ja', 'shinan', 'non']
['¿', 'maton', 'wishabo', '?']

[elaborando, los, carteles, de, los, sectores, del, au

In [19]:
main()

4769it [00:03, 1296.89it/s]
608it [00:00, 1263.45it/s]
608it [00:00, 926.07it/s] 
100%|██████████| 5377/5377 [00:00<00:00, 81072.01it/s]


size of vocab: Vocabulary with namespaces:
 	Non Padded Namespaces: {'*tags', '*labels'}
 	Namespace: tokens, Size: 545 
 	Namespace: target_tokens, Size: 404 

['itanribi', 'itan']
['jatiribi', 'jake', 'westiora', 'itan', 'jake', 'jawen', 'betan', 'wisha']

['biwe', 'jatibi', 'benbo', 'bakebaon', 'itan', 'ainbo', 'bakebaon', 'jawen', 'itan']
['biwe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebaon', 'shinanbo', 'jawen', 'oinxon']

['benbo', 'bakebo', 'itan', 'ainbo', 'bakebo', 'betan', 'aki', 'bokanai', 'itan', 'itan']
['jato', 'shinankinwe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo', 'axetiainko', 'jatibi', 'jan', 'axeti', 'jakon', 'akin', 'ixon', 'itanribi', 'ja', 'akanabo', 'itan', 'ixon']

['yokawe', 'benbo', 'bake', 'itan', 'ainbo', 'bakebo']
['yokawe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo']

['benbo', 'bakebo', 'itan', 'ainbo', 'bakebora', 'axekanai', 'jaska', 'akin', 'atikoma', 'jawekibo', 'mesko', 'jaya', 'teeti', 'itanribi', 'mas', 'que', 'itan', 'que', 'axon']
['benb

['jaskara', 'jisa', 'itanribi', 'jaskaaxon', 'wishati', 'akinbo', 'akax', 'itanribi', 'mesko', 'joibo', 'axon']
['ja', 'wisha', 'ikainko', 'jawekibo', 'wisha', 'itanribi', 'mesko', 'akinbo', 'yoiya']

['tsinkiwe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo', 'westiora', 'axon']
['tsinkiwe', 'jatibi', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebo', 'westiora', 'axon', 'jainxon', 'jato', 'awe']

['shinana', 'ikainko', 'jaton', 'janebo', 'itan', 'wishati', 'itan', 'yoyo', 'ati', 'kirikabo']
['westiora', 'akai', 'jane', 'itan']

['nonra', 'onanti', 'jake', 'nato', 'benbo', 'bakebo', 'itan', 'axekanai', 'jakoni', 'jaton', 'itanribi', 'meskokeska', 'akinbo', 'jato', 'keska', 'akin']
['nonra', 'onanti', 'jake', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebora', 'axekanai', 'jakoni', 'jaton', 'oinax', 'itanribi', 'itanribi', 'jato', 'netetibi']

['jain']
['ja', 'bebon', 'itan', 'ja', 'pekao']

['bokanai', 'westiora', 'jainoax', 'axeti', 'xobo', 'jakonshaman', 'itanribi', 'jakon', 'akin', 'rao', 'a

['oinxon', 'ixon', 'jaton', 'wisha', 'betan', 'ja', 'taxnanti']
['itan', 'oinkin', 'wisha', 'itina', 'betan', 'kirika']

['itan']
['jain', 'iti', 'akana', 'itan', 'jainoax', 'jakoni', 'raenanti']

['jaskaaxon', 'bakebo', 'kopi']
['westiora', 'awe', 'benbo', 'bakebo', 'itan', 'ainbo', 'bakebaon', 'kopi', 'jain', 'jaton', 'jawekibo']

['yoyo', 'awe', 'ja', 'wisha', 'tee', 'itanribi', 'yoikanti', 'ja', 'teebo']
['koshishaman', 'akin', 'jato', 'yoiwe', 'jaskatax', 'teekanti', 'kopi', 'kopi']

['itan', 'jatiribi']
['wishawe', 'itanribi', 'yoi', 'jato', 'oinkanai', 'ixon']

['benbo', 'bake', 'itan', 'ainbo', 'bakebo', 'betan', 'yoyo', 'iwe', 'itan']
['benbo', 'bake', 'itan', 'ainbo', 'bakebo', 'betan', 'yoyo', 'iwe', 'seneanti', 'jaskatax', 'jakoni', 'kopi', 'itanribi', 'jakonax', 'jati']

['itan']
['wishaabo', 'itanribi', 'ja', 'jisa', 'sikayabo']

['itan', 'ati', 'kopi', 'jaton', 'itan', 'jaton']
['nato', 'itan', 'de', 'mia', 'shinanxon', 'non', 'aka', 'itanribi', 'jaskaakin', 'baritia', '

['meniwe', 'jato', 'westiora', 'yokawe', 'jaskaaxon', 'akanti', 'kopi', 'itanribi', 'ja']
['ja', 'tsama', 'ikanabo', 'wisha', 'jainxon', 'jato', 'yoiwe', 'akanon', 'akin', 'jaton', 'jane']

['itan', 'shinanwe', 'jaskaaxon', 'atibo']
['jaskaaxon', 'jakon', 'akin', 'tee', 'atibo', 'awe']

['ja', 'yoikanaibo']
['jaton', 'mia', 'yoiyaibo']

['jaska', 'akin', 'yoyo', 'ati', 'kirika']
['jaskaaxon', 'ati', 'kopi', 'jaskaaxon', 'wisha']

['¿', 'jawekeskaaxonmein', 'non', 'ati', 'iki', 'non', 'ixon', '?']
['¿', 'jawekeskaaxonmein', 'non', 'ati', 'iki', 'akin', 'ixon', 'jayakax', 'teeti', 'kai', 'ixon', '?']

['ikanti', 'jake', ',']
['ikanti', 'jake']

['chonka', 'kaman', 'mesko', 'jawekibo', 'tsinkixon']
['jakon', 'akin', 'ja', 'jawekibo', 'noa', 'jawekibo']

['bakebaon', 'itan']
['bakebaon', 'jain', 'jaton', 'kirika', 'ati']

['itan']
['jainxon', 'jaskaakin']

['jaska', 'akin']
['iwe', 'iki']

['ja', 'mia']
['jato', 'westiora', 'westiorabo', 'yoipakewe']

['yoiwe', 'jato', 'ja', 'jaskaaxon', '

['janebo', 'ja', 'reken', 'senen']
['wishakanai', 'ani', 'kirikain', 'jaskatax', 'jakoni', 'raenanti', 'iki', 'ika', 'shinan', 'akanabo']

['¿', 'jawe', 'jawekibomein', 'non', 'ati', 'iki', 'nato', '?']
['¿', 'jawe', 'jawekiboki', 'non', 'ati', 'iki', 'tsinkikin', 'non', 'axetiain', '?']

['¿', 'yoyo', 'aki', 'bokanai']
['ramara', 'mato', 'ja']

['¿', 'jawekeskaaxonki', 'non', 'akai', '?']
['¿', 'jawekeskaaxonmein', 'non', 'iki', '?']

['nato', 'non', 'aka', 'ja']
['nato', 'jake', 'icha', 'ikaxbi', 'unidad', 'jaya', 'chonka', 'jan']

['jaskara', 'kopira']
['jain']

['jaton', 'jane', 'ani']
['kirika', 'pei', 'bocho', 'jaskaaxon', 'akana', 'jaton', 'jain', 'kirikabo']

['jaskaaxon', 'non', 'jakon', 'akin', 'ati', 'kopi']
['onankanti', 'kopi', 'jawen', 'noa', 'jatikaxbi', 'senen', 'ixon', 'onankanti', 'kopi']

['kointo', 'kointo']
['yoiwe', 'ja', 'kirikain', 'ika', 'joni', 'yoiyaibo', 'ixon']

['papelote', 'jaskaaxon', 'shinankana', 'tee', 'ati', 'jawekibo']
['papelote', 'jaskaaxon', 'ati

['itan', 'jawekeska', 'shinanbo', 'iki']
['yoiwe', 'jaskaakin', 'toponti', 'jawekibo', 'shinanxon', 'ati']

['wishawe', 'ja', 'jane', 'ati']
['wishawe', 'ja', 'jane']

['ja', 'pekao', 'yoiwe', 'jaweranoki']
['ja', 'pekao', 'jato', 'yoiwe', 'tsinkixon', 'akanti', 'kopi']

['jaskaaxon', 'onankanti', 'kopi', 'jaweboki', 'akai', 'ixon']
['jaskaaxon', 'ati', 'kopi', 'kopi', 'wishayabo', 'wetsabaon']

['jaskaaxon', 'jatixonbi', 'min', 'kopi']
['jaskaaxon', 'shinanbo', 'peoti', 'kopi', 'westiora', 'unidad', 'ikainko', 'bakebaon', 'jaton', 'onana', 'jawekibo', 'yoikanti', 'kopi']

['nato', 'jawekibo']
['jaskaaxon', 'non', 'peoti', 'kopira']

['atikoma', 'jawekibo', 'senen', 'akanai', 'jaskaaxon', 'westiora', 'ati', 'kopi']
['atikoma', 'jawekibora', 'non', 'aki', 'kai', 'mesko', 'shinanbo', 'axon']

['jawen']
['japekao']

['¿', 'jawekeskaaxonmein', '?']
['¿', 'jawekeskaaxonmein', 'akanai', '?']

['yoiwe', 'jato', 'tsama', 'ixon', 'jaskatax', 'teeti', 'kopi']
['shinanbo', 'meniwe', 'jaskatax', '

['ani', 'wisha', 'seneanti', 'jawekibo', 'jatixonbi', 'oinkanti', 'kopi']
['wishakana', 'akai', 'jatixonbi', 'yoyo']

['jawen', 'jane', 'kimisha', 'ika', 'iken']
['kimisha', 'jaton', 'jane']

['seneanti', 'jawekibo', 'jato']
['jaskatax', 'teeti', 'shinan', 'awe']

['jato', 'westiora', 'jawekibo']
['yoikanti', 'jawen', 'jawe', 'jawekibo', 'jainxon', 'yoikanti', 'jain', 'ixon']

['shinanmawe', 'rabe', 'jaweki', 'nato']
['jato', 'rabe', 'tee', 'maton', 'aka', 'nato', 'sesion', 'ikainko']

['nonra', 'itanribi', 'non', 'jake', 'ja']
['shinanxon', 'bakebaon', 'tee', 'akanabo', 'mesko', 'tsinkibo', 'axon', 'atikoma', 'jawekibo']

['awe', 'itan', 'shinanwe', 'jaskaaxon', 'ati', 'jawekibo', 'de', 'iketian']
['shinanai', 'jaskaakin', 'nomirobo', 'aka', 'jakon', ',']

['¿', '?']
['¿', 'yoikanti', 'jisa', 'jakon', 'iki', '?']

['yoyo', 'ikai', 'iti', 'kopi']
['wetsa', 'jonibo', 'jan', 'yoyo', 'akai']

['yoyo', 'yoyo', 'akai']
['nenora', 'non', 'kai', 'icha']

['jain', 'kirika', 'jaweki']
['jain', 

['¿', 'jawekeskaki', 'mato', 'shinan', 'yoyo', 'iki', '?']
['¿', 'nenora', '?', 'westiora']

['¿', 'ja', 'wishain']
['¿', 'westiora', 'peokota']

['ixon']
['westiora']

['¿', '?']
['¿', '?', 'westiora']

['jaton']
['oinai', 'oinai', 'bebon', 'westiora', '.', '.']

['mesko', 'yokabo']
['nato', 'yokabo', 'awe']

['ja', 'axon', 'akai', 'primer']
['nato', 'reken', 'primer', 'grado', ',']

['jaton', 'akinwe']
['min']

[]
['nato', 'shinanbo']

['ja', 'yokabo']
['nato', 'yokabo', 'ikainko']

['nato', 'reken', 'unidain']
['nato', 'unidain']

['nato']
['nato', 'axetiainko', ',']

['iamax', '?']
['iamax', '?']

['joibo', 'iamax', 'wishakana']
['wisha', 'pakexbo', 'iamax', 'wisha']

['jaton', 'jane', 'westiora', 'iamax']
['maton', 'shinan', 'akabo', 'iamax', 'ikainko']

['iamax', 'westiora']
['min', 'iamax', 'min', 'bena', 'shinan', 'min']

['axeti', 'iamax']
['axeti', 'chicho', 'iamax', 'jeman']

['tsinkiti']
['tsinkiti']

['tsinkiti']
['tsinkiti']

['tsinkiti']
['tsinkiti']

['tsinkiti']
['tsin

/home/krivas/anaconda3/lib/python3.7/site-packages/allennlp/nn/beam_search.py:127: RuntimeWarning: Empty sequences predicted. You may want to increase the beam size or ensure your step function is working properly.
  RuntimeWarning)


# Religioso

In [17]:
!python training.py --name_file=Religioso/religioso  --dir_files=ParallelCorpora --dir_results=results_religioso_{n_exp}/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
10021it [00:12, 821.72it/s]
1263it [00:01, 736.88it/s]
1263it [00:01, 862.44it/s]
100%|██████████████████████████████████| 11284/11284 [00:00<00:00, 51482.49it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 5.2408 ||: 100%|████████████████████████| 101/101 [00:25<00:00,  4.17it/s]
BLEU: 0.0114, loss: 4.9708 ||: 100%|████████████| 13/13 [00:03<00:00,  3.31it/s]
loss: 4.8852 ||: 100%|████████████████████████| 101/101 [00:25<00:00,  5.01it/s]
BLEU: 0.0051, loss: 5.0493 ||: 100%|████████████| 13/13 [00:08<00:00,  1.26it/s]
loss: 4.7380 ||: 100%|████████████████████████| 101/101 [00:25<00:00,  4.13it/s]
BLEU: 0.0033, loss: 4.6823 ||: 100%|████████████| 13/13 [00:09<00:00,  1.24it/s]
loss: 4.5571 ||: 100%|████████████████████████| 101/101 [00:25<00:00,  3.53it/s]
BLEU: 0.0146, loss: 4.5271 ||: 100%|████████████| 13/13 [00:05<00:00,  1.72i

In [4]:
# Shit
!python training.py --name_file=Religioso/religioso  --dir_files=ParallelCorpora --dir_results=results_religioso_{n_exp}_s/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
10021it [00:12, 817.81it/s]
1263it [00:01, 775.21it/s]
1263it [00:01, 854.98it/s]
100%|██████████████████████████████████| 11284/11284 [00:00<00:00, 51985.78it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 6.1869 ||: 100%|████████████████████████| 101/101 [00:28<00:00,  3.65it/s]
BLEU: 0.0000, loss: 5.8738 ||: 100%|████████████| 13/13 [00:03<00:00,  3.73it/s]
loss: 5.6974 ||: 100%|████████████████████████| 101/101 [00:28<00:00,  4.54it/s]
BLEU: 0.0042, loss: 5.5803 ||: 100%|████████████| 13/13 [00:03<00:00,  3.15it/s]
loss: 5.4699 ||: 100%|████████████████████████| 101/101 [00:29<00:00,  3.69it/s]
BLEU: 0.0072, loss: 5.4390 ||: 100%|████████████| 13/13 [00:04<00:00,  2.56it/s]
loss: 5.3001 ||: 100%|████████████████████████| 101/101 [00:29<00:00,  3.08it/s]
BLEU: 0.0074, loss: 5.3317 ||: 100%|████████████| 13/13 [00:03<00:00,  3.18i

# Flashcards

In [18]:
!python training.py --name_file=Flashcards/flashcards  --dir_files=ParallelCorpora --dir_results=results_flashcards_{n_exp}/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
4876it [00:01, 3143.13it/s]
774it [00:00, 3128.02it/s]
2090it [00:00, 2872.60it/s]
100%|███████████████████████████████████| 5650/5650 [00:00<00:00, 150647.89it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 3.4053 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 14.35it/s]
BLEU: 0.0000, loss: 2.9127 ||: 100%|██████████████| 8/8 [00:01<00:00,  4.34it/s]
loss: 2.7876 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 15.21it/s]
BLEU: 0.0000, loss: 2.6649 ||: 100%|██████████████| 8/8 [00:00<00:00, 11.57it/s]
loss: 2.5748 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.43it/s]
BLEU: 0.0508, loss: 2.5563 ||: 100%|██████████████| 8/8 [00:00<00:00, 11.11it/s]
loss: 2.3764 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 15.05it/s]
BLEU: 0.0623, loss: 2.3252 ||: 100%|██████████████| 8/8 [00:00<00:00, 10.81

In [5]:
#Shit
!python training.py --name_file=Flashcards/flashcards  --dir_files=ParallelCorpora --dir_results=results_flashcards_{n_exp}/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
4876it [00:01, 2605.42it/s]
774it [00:00, 2619.60it/s]
2090it [00:00, 2331.36it/s]
100%|████████████████████████████████████| 5650/5650 [00:00<00:00, 89411.56it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 4.2122 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.98it/s]
BLEU: 0.1473, loss: 3.7588 ||: 100%|██████████████| 8/8 [00:00<00:00, 10.58it/s]
loss: 3.6128 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.08it/s]
BLEU: 0.0000, loss: 3.4313 ||: 100%|██████████████| 8/8 [00:00<00:00, 10.42it/s]
loss: 3.3220 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 12.98it/s]
BLEU: 0.0000, loss: 3.2367 ||: 100%|██████████████| 8/8 [00:00<00:00,  8.92it/s]
loss: 3.0924 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 14.60it/s]
BLEU: 0.0274, loss: 3.0916 ||: 100%|██████████████| 8/8 [00:00<00:00,  8.06

In [7]:
#Shit
!python training.py --name_file=Flashcards/flashcards  --dir_files=ParallelCorpora --dir_results=results_flashcards_6/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
4876it [00:02, 2309.86it/s]
774it [00:00, 2593.11it/s]
2090it [00:00, 2306.91it/s]
100%|████████████████████████████████████| 5650/5650 [00:00<00:00, 88279.10it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 4.2121 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.96it/s]
BLEU: 0.1473, loss: 3.7587 ||: 100%|██████████████| 8/8 [00:00<00:00, 11.28it/s]
loss: 3.6111 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.04it/s]
BLEU: 0.0000, loss: 3.4325 ||: 100%|██████████████| 8/8 [00:00<00:00,  9.95it/s]
loss: 3.3051 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 13.00it/s]
BLEU: 0.0000, loss: 3.3111 ||: 100%|██████████████| 8/8 [00:00<00:00,  8.61it/s]
loss: 3.1052 ||: 100%|██████████████████████████| 49/49 [00:03<00:00, 14.32it/s]
BLEU: 0.0000, loss: 3.0978 ||: 100%|██████████████| 8/8 [00:00<00:00,  8.92

# All

In [6]:
#Shit
!python training.py --name_file=All/all  --dir_files=ParallelCorpora --dir_results=results_all_{n_exp}/bpe_5000 --evaluate=False  --bpe=False

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
16549it [00:13, 1199.30it/s]
2627it [00:02, 1160.13it/s]
7093it [00:05, 1224.70it/s]
100%|██████████████████████████████████| 19176/19176 [00:00<00:00, 69152.51it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 5.6590 ||: 100%|████████████████████████| 166/166 [00:36<00:00,  4.55it/s]
BLEU: 0.0009, loss: 5.2716 ||: 100%|████████████| 27/27 [00:17<00:00,  1.25it/s]
loss: 4.9150 ||: 100%|████████████████████████| 166/166 [00:36<00:00,  4.37it/s]
BLEU: 0.0071, loss: 4.9004 ||: 100%|████████████| 27/27 [00:06<00:00,  3.35it/s]
loss: 4.6435 ||: 100%|████████████████████████| 166/166 [00:37<00:00,  5.43it/s]
BLEU: 0.0099, loss: 4.6323 ||: 100%|████████████| 27/27 [00:06<00:00,  3.11it/s]
loss: 4.2919 ||: 100%|████████████████████████| 166/166 [00:37<00:00,  5.60it/s]
BLEU: 0.0103, loss: 4.4620 ||: 100%|████████████| 27/27 [00:07<00:00,  2.